<a href="https://colab.research.google.com/github/razamehar/PyTorch/blob/main/PyTorch_04_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Question-Answer using RNN in PyTorch.**

---



# UTILITY

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [ ]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [ ]:
# vocab
vocab = {'<UNK>':0}

def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)

In [ ]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

# DATA DOWNLOAD & REVIEW

In [ ]:
df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


# DATA PREPROCESSING

In [ ]:
df.apply(build_vocab, axis=1)
len(vocab)

324

# DATASET CLASS

In [ ]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [ ]:
# Create dataset object
dataset = QADataset(df, vocab)

In [ ]:
# Create dataloader object
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# NEURAL NETWORK CLASS

In [ ]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [ ]:
learning_rate = 0.001
epochs = 20

model = SimpleRNN(len(vocab))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# TRAINING

In [ ]:
# Training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Total Loss: {total_loss:4f}")

Epoch: 1, Total Loss: 9.277322
Epoch: 2, Total Loss: 8.240559
Epoch: 3, Total Loss: 7.299380
Epoch: 4, Total Loss: 6.499185
Epoch: 5, Total Loss: 5.856381
Epoch: 6, Total Loss: 5.242877
Epoch: 7, Total Loss: 4.763370
Epoch: 8, Total Loss: 4.332192
Epoch: 9, Total Loss: 3.964645
Epoch: 10, Total Loss: 3.620976
Epoch: 11, Total Loss: 3.337424
Epoch: 12, Total Loss: 3.065990
Epoch: 13, Total Loss: 2.830522
Epoch: 14, Total Loss: 2.625667
Epoch: 15, Total Loss: 2.432938
Epoch: 16, Total Loss: 2.260090
Epoch: 17, Total Loss: 2.104709
Epoch: 18, Total Loss: 1.961063
Epoch: 19, Total Loss: 1.830962
Epoch: 20, Total Loss: 1.711154


# MODEL EVALUATION

In [ ]:
def predict(model, question, threshold=0.7):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [ ]:
predict(model, "What is campus x?")

I don't know
asia
